# Pandas

In [5]:
import pandas as pd

df = pd.read_csv('studietimmar_poang.csv')
df.columns = (df.columns
              .str.strip()
              .str.replace(' ','_')
              .str.lower()
              )

df.head()

,student,studietimmar,provpoäng,klass
0,Elev 065,0.7,55.0,B
1,Elev 059,NaN,94.0,A
2,Elev 060,8.0,100.0,A
3,Elev 005,5.8,100.0,A
4,Elev 070,NaN,93.0,B


In [9]:
print(df.info(),'\n')
print(df.isna().sum(), '\n')
print(df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   student       80 non-null     object 
 1   studietimmar  76 non-null     float64
 2   provpoäng     78 non-null     float64
 3   klass         80 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.6+ KB
None 

student         0
studietimmar    4
provpoäng       2
klass           0
dtype: int64 

student          object
studietimmar    float64
provpoäng       float64
klass            object
dtype: object


In [18]:
df_clean = df.dropna(subset=['provpoäng'])
df_clean.loc[df_clean['studietimmar'].isna(), 'studietimmar'] = 0
df_clean.head()

,student,studietimmar,provpoäng,klass
0,Elev 065,0.7,55.0,B
1,Elev 059,0.0,94.0,A
2,Elev 060,8.0,100.0,A
3,Elev 005,5.8,100.0,A
4,Elev 070,0.0,93.0,B


In [22]:
df_clean.isnull().sum()


student         0
studietimmar    0
provpoäng       0
klass           0
dtype: int64

In [32]:
print(f'Dubletter: {df_clean.duplicated().sum()}')
df_clean = df_clean.drop_duplicates().reset_index(drop=True)
print(f'Unika studenter: {df_clean['student'].nunique()}')
df_clean.head()

Dubletter: 0
Unika studenter: 78


,student,studietimmar,provpoäng,klass
0,Elev 065,0.7,55.0,B
1,Elev 059,0.0,94.0,A
2,Elev 060,8.0,100.0,A
3,Elev 005,5.8,100.0,A
4,Elev 070,0.0,93.0,B


In [33]:
df_clean['klass'] = df_clean['klass'].astype('category')
df_clean.dtypes

student           object
studietimmar     float64
provpoäng        float64
klass           category
dtype: object

In [35]:
# df_clean['datum'] = pd.to_datetime(df_clean['datum'], dayfirst=True, errors='coerce')

In [40]:
df_clean = df_clean.assign(
    poang_per_timme = df_clean['provpoäng']
    .where(df_clean['studietimmar'] > 0, other=pd.NA)
    / df_clean['studietimmar']
)
df_clean[['student', 'klass', 'studietimmar', 'provpoäng', 'poang_per_timme']].head()

,student,klass,studietimmar,provpoäng,poang_per_timme
0,Elev 065,B,0.7,55.0,78.571429
1,Elev 059,A,0.0,94.0,NaN
2,Elev 060,A,8.0,100.0,12.500000
3,Elev 005,A,5.8,100.0,17.241379
4,Elev 070,B,0.0,93.0,NaN


In [47]:
summary_klass = (df_clean.
                 groupby('klass', dropna=False, observed=True)
                 .agg(
                     anal = ('student', 'count'),
                     medel_poäng = ('provpoäng', 'mean'),
                     median_poäng = ('provpoäng', 'median'),
                     std_poäng = ('provpoäng', 'std'),
                     medel_timmar = ('studietimmar', 'mean')
                 ).reset_index()
                 )

summary_klass

,klass,anal,medel_poäng,median_poäng,std_poäng,medel_timmar
0,A,25,85.720000,89.0,11.900700,5.184000
1,B,27,73.148148,70.0,13.863909,4.685185
2,C,26,71.307692,70.0,9.926809,3.915385


In [49]:

bins = [0, 60, 80, 100]
andelar = pd.crosstab(
    df_clean['klass'],
    pd.cut(df_clean['provpoäng'], bins=bins, right=True), 
    normalize='index'
).round(2)

andelar

provpoäng,"(0, 60]","(60, 80]","(80, 100]"
klass,,,
A,0.00,0.40,0.60
B,0.19,0.59,0.22
C,0.15,0.69,0.15


In [56]:
assert df_clean['provpoäng'].between(0, 100).all()
assert (df_clean['studietimmar'] >= 0).all()

corr = df_clean[['studietimmar', 'provpoäng']].corr().iloc[0,1]
print(f'Korrelation timmar-poäng: {round(float(corr), 2)}')

Korrelation timmar-poäng: 0.59


In [58]:
df_clean.to_csv('resultat_städat.csv', index=False)
summary_klass.to_csv('summary_by_class.csv', index=False)
andelar.to_csv('summary_by_class_bins.csv')